In [1]:
import pickle
import pandas as pd
import gensim
from gensim.models import KeyedVectors
import networkx as nx

In [3]:
experiment_names = ['NTLM_100_epochs', '2_10', '2_18', '2_18_100_epochs', '2_18_500_epochs', '2_18_500_epochs_v2']

def operator_hadamard(u, v):
        return u * v

for name in experiment_names:
    with open(f'../lm-vol/{name}_lr.pkl', 'rb') as file:
        clf = pickle.load(file)
    model = gensim.models.Word2Vec.load(f'../lm-vol/{name}_word2vec.model')
    wv = KeyedVectors.load(f'../lm-vol/{name}_word2vec.wordvectors', mmap='r')
    graph_structure = ('source', 'source_computer', 'destination')
    inconclusive = 0
    n = 0
    red = pd.read_csv('../lm-vol/redteam_complete.csv')
    client_to_ip_pred = []
    ip_to_service_pred = []
    for index, row in red.iterrows():
        try: 
            client_embedding = wv[(row[graph_structure[0]])]
            ip_embedding = wv[(row[graph_structure[1]])]
            service_embedding = wv[(row[graph_structure[2]])]
            client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
            ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
            client_to_ip = clf.predict_proba(client_to_ip_embedding)
            ip_to_service = clf.predict_proba(ip_to_service_embedding)
            client_to_ip_pred.append(client_to_ip[0][1])
            ip_to_service_pred.append(ip_to_service[0][1])
        except: 
            client_to_ip_pred.append(None)
            ip_to_service_pred.append(None)
            inconclusive += 1
    red['client_to_ip_pred'] = client_to_ip_pred
    red['ip_to_service_pred'] = ip_to_service_pred
    red_anomalies = red[(red["ip_to_service_pred"].astype(float) <= 0.1) | (red["client_to_ip_pred"].astype(float) <= 0.1)]
    red_invalids = red[(red["ip_to_service_pred"] == None) & (red["client_to_ip_pred"] == None)]
    display(red_anomalies)
    display(red)
    test_G = nx.read_graphml('../lm-vol/LANL_test.graphml')
    test = []
    # max = 100
    for index, edge in enumerate(test_G.edges):
    #     if index > max:
    #         break
        try:
            src = wv[edge[0]]
            dest = wv[edge[1]]
            src_to_dest = [operator_hadamard(src, dest)]
            prob = clf.predict_proba(src_to_dest)
            test.append(prob[0][1])
        except:
            test.append(None)
    anomalies = 0
    for prob in test:
        if prob is not None and prob <= 0.1:
            anomalies+=1
    print(anomalies)
#     graph_structure = (1, 3, 2)
#     client_to_ip_pred = []
#     ip_to_service_pred = []
#     inconclusive = 0
#     for chunk in pd.read_csv('../lm-vol/LANL_test.csv', header=None, chunksize=1000000):
#     #     display(chunk)
#         for index, row in chunk.iterrows():
#             try: 
#                 client_embedding = wv[(row[graph_structure[0]])]
#                 ip_embedding = wv[(row[graph_structure[1]])]
#                 service_embedding = wv[(row[graph_structure[2]])]
#                 client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
#                 ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
#                 client_to_ip = clf.predict_proba(client_to_ip_embedding)
#                 ip_to_service = clf.predict_proba(ip_to_service_embedding)
#                 client_to_ip_pred.append(client_to_ip[0][1])
#                 ip_to_service_pred.append(ip_to_service[0][1])
#             except: 
#                 client_to_ip_pred.append(None)
#                 ip_to_service_pred.append(None)
#                 inconclusive += 1
#         chunk['client_to_ip_pred'] = client_to_ip_pred
#         chunk['ip_to_service_pred'] = ip_to_service_pred
#         chunk_anomalies = chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)]
#         chunk_invalids = chunk[(chunk["ip_to_service_pred"] == None) & (chunk["client_to_ip_pred"] == None)]
#         display(chunk_anomalies)
#         break
    break

,Unnamed: 0,time,source,source_computer,destination_computer,destination,authentication_type,logon_type,authentication_orientation,Success,client_to_ip_pred,ip_to_service_pred
33,33,464545,U737@DOM1,C19932,C612,U737@DOM1,NTLM,Network,LogOn,Success,0.000504,0.000504
34,34,466195,U737@DOM1,C19932,C457,U737@DOM1,NTLM,Network,LogOn,Success,0.000504,0.000504
46,46,578438,U737@DOM1,C19932,C1065,U737@DOM1,NTLM,Network,LogOn,Success,0.000504,0.000504
48,48,584564,U737@DOM1,C19932,C612,U737@DOM1,NTLM,Network,LogOn,Success,0.000504,0.000504
49,49,635015,U737@DOM1,C19932,C529,U737@DOM1,NTLM,Network,LogOn,Success,0.000504,0.000504
484,484,1089597,U737@DOM1,C19932,C586,U737@DOM1,NTLM,Network,LogOn,Success,0.000504,0.000504
549,549,1165480,U737@DOM1,C19932,C625,U737@DOM1,NTLM,Network,LogOn,Success,0.000504,0.000504
649,649,1328198,U737@DOM1,C19932,C467,U737@DOM1,NTLM,Network,LogOn,Success,0.000504,0.000504
675,675,1758226,U737@DOM1,C19932,C467,U737@DOM1,NTLM,Network,LogOn,Success,0.000504,0.000504
680,680,1931895,U737@DOM1,C19932,C612,U737@DOM1,NTLM,Network,LogOn,Success,0.000504,0.000504


,Unnamed: 0,time,source,source_computer,destination_computer,destination,authentication_type,logon_type,authentication_orientation,Success,client_to_ip_pred,ip_to_service_pred
0,0,150885,U620@DOM1,C17693,C1003,U620@DOM1,NTLM,Network,LogOn,Success,NaN,NaN
1,1,151036,U748@DOM1,C17693,C305,U748@DOM1,NTLM,Network,LogOn,Success,NaN,NaN
2,2,151648,U748@DOM1,C17693,C728,U748@DOM1,NTLM,Network,LogOn,Success,NaN,NaN
3,3,151993,U6115@DOM1,C17693,C1173,U6115@DOM1,NTLM,Network,LogOn,Success,NaN,NaN
4,4,153792,U636@DOM1,C17693,C294,U636@DOM1,NTLM,Network,LogOn,Success,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
734,734,2552687,U655@DOM1,C17693,C828,U655@DOM1,NTLM,Network,LogOn,Success,NaN,NaN
735,735,2552687,U655@DOM1,C17693,C828,U655@DOM1,NTLM,Network,LogOn,Success,NaN,NaN
736,736,2552687,U655@DOM1,C17693,C828,U655@DOM1,NTLM,Network,LogOn,Success,NaN,NaN
737,737,2552687,U655@DOM1,C17693,C828,U655@DOM1,NTLM,Network,LogOn,Success,NaN,NaN


95178


In [4]:
print(len(test_G.edges))

515636


In [5]:
import dask.dataframe as dd

In [6]:
unique_test_df = dd.read_csv('../lm-vol/LANL_test_unique.csv', dtype='str', blocksize="1024MB")
print(unique_test_df.shape[0].compute())
unique_test_df.head(100)

1062874


,source user@domain,destination computer,source computer
0,ANONYMOUS LOGON@C586,C586,C1250
1,ANONYMOUS LOGON@C586,C586,C586
2,C101$@DOM1,C988,C988
3,C1020$@DOM1,C1020,C1020
4,C1021$@DOM1,C625,C1021
...,...,...,...
95,C688$@DOM1,C1065,C688
96,C693$@DOM1,C528,C528
97,C725$@DOM1,C725,C725
98,C740$@DOM1,C740,C740
